In [1]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO
import time
from datetime import date, datetime, timedelta
import random
import os

In [11]:
# Import packages
import yfinance as yf

# Read and print the stock tickers that make up S&P500
df_tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
df_tickers = df_tickers.rename(columns={'Symbol':'ticker'})
# Make sure GOOG and GOOGL are not both included
df_tickers = df_tickers[df_tickers['ticker'] != 'GOOG']

try:
    df_available = pd.read_csv('data/data.csv')
    df_tickers = df_tickers[~df_tickers['ticker'].isin(df_available['ticker'])]
except:
    pass

random.seed(1993)
# Get stratified random 17 samples per sector
# df_tickers_sample = df_tickers.groupby('GICS Sector', group_keys=False).apply(lambda x: x.sample(17))
df_tickers_sample = df_tickers.sample(230)

tickers = list(df_tickers_sample['ticker'])

# Custom Tickers
# tickers.extend(['BTC/USD', 'ETH/USD', 'XRP/USD', 'LTC/USD', 'ADA/USD', 'MATIC/USD'])

C:\Users\18327\AppData\Local\Temp\ipykernel_8980\2910404954.py:11: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_available = pd.read_csv('data/data.csv')


In [13]:
# Twelve Data
api_key = twelve_data_api

data_frames = []

for ticker in tickers:
        
    try:
        # Get Earliest Date
        url = f'https://api.twelvedata.com/earliest_timestamp?symbol={ticker}&interval=1day&apikey={api_key}'
        r = requests.get(url)
        
        start_date = r.text.split('"')[3]
        end_date = date.today().strftime('%Y-%m-%d')
        total_days = (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days
        
        batch_size = np.ceil(total_days / 5000).astype(int)
        
        l_dates = []      
        
        # If multiple batches, append start and end dates for every 5000 days
        if batch_size > 1:  
            for i in range(batch_size-1):
                l_dates.append((pd.to_datetime(end_date)  - timedelta(days=(5000 * i))).strftime('%Y-%m-%d'))
            l_dates.append(start_date)
        else:
            l_dates.append(end_date)
            l_dates.append(start_date)
        
        # Check how many credits are left, and pause for 60 seconds to reset Twelve Data's limit
        api_credit = r.headers['Api-Credits-Left']
        if api_credit == '0':
            time.sleep(61)
        
        for i in range(batch_size):
            try:
                batch_end = l_dates[i]
                batch_start = l_dates[i+1]
                
                # Make sure date range don't overlap throughout the loop
                if (i > 0):
                    batch_end = (pd.to_datetime(batch_end) - timedelta(days=1)).strftime('%Y-%m-%d')
                
                url = f'https://api.twelvedata.com/time_series?&start_date={batch_start}&end_date={batch_end}&symbol={ticker}&format=CSV&interval=1day&apikey={api_key}'
                r = requests.get(url)

                # Decode bytes into a string
                data_string = r.content.decode('utf-8')

                # Use StringIO to treat the string as a file-like object
                data_file = BytesIO(data_string.encode())
                
                # Union the new data to data from previous iteration. 
                data = pd.read_csv(data_file, delimiter=';')
                data['ticker'] = ticker
                
                # Append data to list
                data_frames.append(data)
                
                # Check how many credits are left, and pause for 60 seconds to reset Twelve Data's limit
                api_credit = r.headers['Api-Credits-Left']
                if api_credit == '0':
                    time.sleep(61)
                
            except:
                pass
        
        print(data_frames[-1])

    except:
        print(f"Couldn't download {ticker}.")

df_final = pd.concat([i for i in data_frames if len(i) > 0])

merge_cols = ['GICS Sector', 'GICS Sub-Industry', 'Headquarters Location', 'Date added', 'Founded'] # 'CIK'
df_model = df_final.merge(df_tickers, on='ticker', how='left')

# Only save tickers with 5 years or more data
ticker_days = df_model.groupby('ticker')['ticker'].value_counts().reset_index()
five_yrs_more = ticker_days[ticker_days['count'] >= 1825]['ticker']
df_model = df_model[df_model['ticker'].isin(five_yrs_more)]

df_model.to_csv(f'data/raw/{datetime.now().strftime("%Y-%m-%d-%H-%M-%S")}.csv', index=False)

        datetime   open      high       low     close  volume ticker
0     1996-12-02  19.15  19.15000  18.85000  19.07000  217200    EFX
1     1996-11-29  19.07  19.29000  19.07000  19.15000  127400    EFX
2     1996-11-27  19.36  19.80000  19.00000  19.00000  392700    EFX
3     1996-11-26  19.29  19.51000  19.22000  19.36000  194000    EFX
4     1996-11-25  18.71  19.51000  18.41000  19.36000  481100    EFX
...          ...    ...       ...       ...       ...     ...    ...
4202  1980-03-21   0.43   0.44000   0.43000   0.43000   12000    EFX
4203  1980-03-20   0.44   0.44000   0.44000   0.44000    2400    EFX
4204  1980-03-18   0.44   0.44000   0.44000   0.44000   19200    EFX
4205  1980-03-17   0.45   0.45000   0.44000   0.45000   57600    EFX
4206  1980-03-17   0.00   0.77604   0.76042   0.76562   57600    EFX

[4207 rows x 7 columns]
        datetime      open      high       low     close   volume ticker
0     1996-12-02  15.17000  15.24000  15.09000  15.20000  1962000    AXP
1

OSError: [Errno 22] Invalid argument: 'data/raw/2024-04-21 02:06:13.csv'

In [19]:
# List files in the directory
directory = 'data/raw/'
files = os.listdir(directory)

l_all_raw = []
for i in files:
    l_all_raw.append(pd.read_csv(f'{directory}{i}'))

df_out = pd.concat(l_all_raw).drop_duplicates(subset=['datetime', 'ticker']) 
ticker_days = df_out.groupby('ticker')['ticker'].value_counts().reset_index()

# Only include tickers with 5 years or more observations
five_yrs_more = ticker_days[ticker_days['count'] >= 1825]['ticker']
df_out = df_out[df_out['ticker'].isin(five_yrs_more)]

df_out.to_csv('data/data.csv', index_label=False)
df_out

C:\Users\18327\AppData\Local\Temp\ipykernel_8980\3583801672.py:7: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  l_all_raw.append(pd.read_csv(f'{directory}{i}'))
C:\Users\18327\AppData\Local\Temp\ipykernel_8980\3583801672.py:7: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  l_all_raw.append(pd.read_csv(f'{directory}{i}'))


,datetime,open,high,low,close,volume,ticker,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,2024-04-19,24.92,25.01,24.72,24.78,490400.0,NWS,News Corp (Class B),Communication Services,Publishing,"New York City, New York",2015-09-18,1564708.0,2013 (News Corporation 1980)
1,2024-04-18,24.93,25.37,24.75,24.81,659100.0,NWS,News Corp (Class B),Communication Services,Publishing,"New York City, New York",2015-09-18,1564708.0,2013 (News Corporation 1980)
2,2024-04-17,25.00,25.09,24.92,24.95,969800.0,NWS,News Corp (Class B),Communication Services,Publishing,"New York City, New York",2015-09-18,1564708.0,2013 (News Corporation 1980)
3,2024-04-16,24.84,24.99,24.67,24.82,531200.0,NWS,News Corp (Class B),Communication Services,Publishing,"New York City, New York",2015-09-18,1564708.0,2013 (News Corporation 1980)
4,2024-04-15,25.35,25.49,24.95,25.00,685800.0,NWS,News Corp (Class B),Communication Services,Publishing,"New York City, New York",2015-09-18,1564708.0,2013 (News Corporation 1980)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1784206,1980-03-20,2.19,2.23,2.19,2.19,387000.0,TT,Trane Technologies,Industrials,Building Products,"Dublin, Ireland",2010-11-17,1466258.0,1871
1784207,1980-03-19,2.24,2.27,2.24,2.24,1092000.0,TT,Trane Technologies,Industrials,Building Products,"Dublin, Ireland",2010-11-17,1466258.0,1871
1784208,1980-03-18,2.26,2.27,2.23,2.26,882000.0,TT,Trane Technologies,Industrials,Building Products,"Dublin, Ireland",2010-11-17,1466258.0,1871
1784209,1980-03-17,2.23,2.30,2.23,2.23,549000.0,TT,Trane Technologies,Industrials,Building Products,"Dublin, Ireland",2010-11-17,1466258.0,1871
